# Formateo de Datos
Como mencionamos anteriormente. Nos aprovecharemos de la forma de la base de datos para darle un formato más adecuado para su análisis. La estructura constará de usar un múlti indice en la parte de las columnas y de renglonres los municipios. 

## Extracción de Datos crudos.

In [117]:
import pandas as pd
import numpy as np

# Lista de Municipios.
Sonora = pd.read_excel('..\\medium_data\\book_son.xlsx',sheet_name=None)
Municipios = list(Sonora.keys())

# Datos a Ordenar.
import indicators_tables
sonora = {municipio: indicators_tables.indicators_tables(Sonora, municipio) for municipio in Municipios}

## Construccion de las Tablas

### Composición Territorial. (1)

In [118]:
mun_0 = Municipios[0]
def base_format(mun, i: int):
    df = sonora[mun][i].copy()
    df.set_index(df.columns[0],inplace=True)
    df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
    return df
df = base_format(mun_0, 1)
df

,Unnamed: 2,Unnamed: 4
Unnamed: 1,,
Composición territorial,Absoluto,Porcentaje
Localidades,25,-
Urbanas,1,4
Rurales,24,96
AGEB,14,-
Urbanas,11,78.6
Rurales,3,21.4
Manzanas,223,-
Urbanas,135,60.5


In [119]:
def compos_format(mun: str):
    df = base_format(mun, 1)
    df.columns = df.iloc[0,:]
    df = df.iloc[1:-1]
    return df


level_zero = ['Localidades', 'AGEB', 'Manzanas']
level_one = ['Total','Urbanas','Rurales']

cols = pd.MultiIndex.from_product([level_zero, level_one],names=[2,2])

def fill_df(cols : pd.MultiIndex | list, adapt):
    abs_df = pd.DataFrame(index = Municipios, columns=cols)
    per_df = pd.DataFrame(index = Municipios, columns=cols)
    for mun in Municipios:
        data = adapt(mun)
        abs_df.loc[mun,:] = list(data['Absoluto'])
        per_df.loc[mun,:] = list(data['Porcentaje'])
    return abs_df, per_df



In [120]:
compos = pd.ExcelWriter("..\\tidy_data\\ComposicionTerritorial.xlsx")

absolute, percent = fill_df(cols, compos_format)
absolute.to_excel(compos, 'Absoluto')
percent.to_excel(compos, 'Porcentaje')

compos.close()

### Ubicación Geográfica (2)

In [121]:
df = sonora[mun_0][2].copy()
def ubigeo_format(mun: str):
    df = base_format(mun, 2)
    df.columns = df.iloc[0,:]
    df = df.iloc[1:-1]
    return df

df = ubigeo_format(mun_0)
df

Ubicación geográfica,Absoluto,Porcentaje
Unnamed: 1,,
Coordenadas,Entre los paralelos 29° 41’ y 29° 53’ de latit...,-
Altitud,Entre 500 y 2 200 metros.,-
Colindancias,Colinda al norte con los municipios de San Fel...,-
Superficie (Km2),368.2,0.2
Densidad de población (habitantes/km2),7,-


En este caso, vamos a separar la tabla en dos partes. La parte geográfica (Coordenadas, Altitud, Colindancias) y la parte númerica (Superficie y Densidad de Población)

#### Parte Geográfica 

In [122]:
geo_df = df.loc[['Coordenadas','Altitud','Colindancias'],:]

geo_zero = ['Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Coordenadas', 'Colindancias']
geo_one = ['Latitud', 'Latitud', 'Longitud', 'Longitud', 'Altitud', 'Altitud', '']
geo_two = ['Min (°)', 'Max (°)', 'Min (°)', 'Max (°)', 'Min (m)', 'Max (m)', '']
cols = pd.MultiIndex.from_arrays([geo_zero, geo_one, geo_two])
geo_son = pd.DataFrame(columns=cols, index = Municipios)
geo_son.iloc[0, 6] = df.loc['Colindancias','Absoluto']
geo_son

Coordenadas                                   \
                                Latitud         Longitud         Altitud   
                                Min (°) Max (°)  Min (°) Max (°) Min (m)   
Aconchi                             NaN     NaN      NaN     NaN     NaN   
Agua Prieta                         NaN     NaN      NaN     NaN     NaN   
Alamos                              NaN     NaN      NaN     NaN     NaN   
Altar                               NaN     NaN      NaN     NaN     NaN   
Arivechi                            NaN     NaN      NaN     NaN     NaN   
...                                 ...     ...      ...     ...     ...   
Villa Pesqueira                     NaN     NaN      NaN     NaN     NaN   
Yécora                              NaN     NaN      NaN     NaN     NaN   
Gral. Plutarco Elías Calles         NaN     NaN      NaN     NaN     NaN   
Benito Juárez                       NaN     NaN      NaN     NaN     NaN   
San Ignacio Río Muerto              NaN     NaN      NaN     NaN     NaN   

                                     \
                                      
                            Max (m)   
Aconchi                         NaN   
Agua Prieta                     NaN   
Alamos                          NaN   
Altar                           NaN   
Arivechi                        NaN   
...                             ...   
Villa Pesqueira                 NaN   
Yécora                          NaN   
Gral. Plutarco Elías Calles     NaN   
Benito Juárez                   NaN   
San Ignacio Río Muerto          NaN   

                                                                  Colindancias  
                                                                                
                                                                                
Aconchi                      Colinda al norte con los municipios de San Fel...  
Agua Prieta                                                                NaN  
Alamos                                                                     NaN  
Altar                                                                      NaN  
Arivechi                                                                   NaN  
...                                                                        ...  
Villa Pesqueira                                                            NaN  
Yécora                                                                     NaN  
Gral. Plutarco Elías Calles                                                NaN  
Benito Juárez                                                              NaN  
San Ignacio Río Muerto                                                     NaN  

[72 rows x 7 columns]

Ahora si, construimos la tablas y la guardamos

In [123]:
import ubigeo as ug

def geo_ubication(i: int, mun: str, uf: pd.DataFrame):
    df = sonora[mun][2].copy()
    df.set_index('Unnamed: 1',inplace=True)
    df = df.replace({'nan': pd.NA}).dropna(axis = 1, how  = 'all').drop(columns = 'NA')
    df.columns = df.iloc[0,:]
    df = df.drop(index = ['Ubicación geográfica','Fuente: INEGI.'])
    uf.iloc[i, 6] = df.loc['Colindancias','Absoluto']
    coords = df.loc['Coordenadas','Absoluto']
    altura = df.loc['Altitud','Absoluto']
    latlonalt = ug.get_latlong(coords) + ug.get_altitud(altura)
    uf.iloc[i, :6] = latlonalt
    # print(mun)
  
for m, mun in enumerate(Municipios):
    geo_ubication(m, mun, geo_son)
geo_son.to_excel('..\\tidy_data\\Geografia.xlsx')
geo_son



Coordenadas                                     \
                                Latitud               Longitud               
                                Min (°)    Max (°)     Min (°)     Max (°)   
Aconchi                       29.683333  29.883333       110.0  110.433333   
Agua Prieta                        30.7      31.35  108.683333      109.75   
Alamos                        26.383333  27.783333  108.416667      109.25   
Altar                         30.516667  31.666667       111.6  112.283333   
Arivechi                      28.733333  28.983333  108.833333  109.266667   
...                                 ...        ...         ...         ...   
Villa Pesqueira               28.783333       29.5  109.783333  110.133333   
Yécora                        28.166667      28.65  108.466667  110.133333   
Gral. Plutarco Elías Calles   31.316667  32.083333       112.3  113.516667   
Benito Juárez                 26.966667  27.233333  109.716667  109.983333   
San Ignacio Río Muerto        27.116667  27.533333  110.133333       110.6   

                                             \
                            Altitud           
                            Min (m) Max (m)   
Aconchi                       500.0  2200.0   
Agua Prieta                   800.0  2500.0   
Alamos                         50.0  2000.0   
Altar                         200.0  1700.0   
Arivechi                      400.0  2200.0   
...                             ...     ...   
Villa Pesqueira                50.0  1700.0   
Yécora                        300.0  2300.0   
Gral. Plutarco Elías Calles   100.0  1400.0   
Benito Juárez                   0.0   100.0   
San Ignacio Río Muerto          0.0   100.0   

                                                                  Colindancias  
                                                                                
                                                                                
Aconchi                      Colinda al norte con los municipios de San Fel...  
Agua Prieta                  Colinda al norte con los Estados Unidos de Amé...  
Alamos                       Colinda al norte con el municipio de Quiriego ...  
Altar                        Colinda al norte con el municipio de Caborca y...  
Arivechi                     Colinda al norte, este, sur y oeste con el mun...  
...                                                                        ...  
Villa Pesqueira              Colinda al norte con los municipios de Baviáco...  
Yécora                       Colinda al norte con los municipios de Bacanor...  
Gral. Plutarco Elías Calles  Colinda al norte con el municipio de Puerto Pe...  
Benito Juárez                Colinda al norte con los municipios de Cajeme ...  
San Ignacio Río Muerto       Colinda al norte con los municipios de Guaymas...  

[72 rows x 7 columns]

#### Parte Númerica

In [124]:
ubi_df = df.iloc[3:,:]
ubi_df

Ubicación geográfica,Absoluto,Porcentaje
Unnamed: 1,,
Superficie (Km2),368.2,0.2
Densidad de población (habitantes/km2),7,-


In [125]:
def ubigeo_adapt(mun):
    y = ubigeo_format(mun)
    df = y.iloc[3:,:]
    return df

ubi_cols = list(ubi_df.index)



In [126]:
ubigeos = pd.ExcelWriter("..\\tidy_data\\UbicacionGeografica.xlsx")

absolute, percent = fill_df(ubi_cols, ubigeo_adapt)
absolute.to_excel(ubigeos, 'Absoluto')
percent.to_excel(ubigeos, 'Porcentaje')

ubigeos.close()

### Dinámica Poblacional (3/4)

En este caso tenemos dos tablas. De igual forma las trabajaremos por separado.

In [139]:
def save_excel(filename: str, cols: pd.MultiIndex|list , ad):
    writer = pd.ExcelWriter(f"..\\tidy_data\\{filename}")
    absolute, percent = fill_df(cols, ad)
    absolute.to_excel(writer, 'Absoluto')
    percent.to_excel(writer, 'Porcentaje')
    writer.close()

def din_pob_adapt1(mun:str):
    y = base_format(mun, 3)
    y.columns = y.iloc[0,:]
    df = y[['Absoluto','Porcentaje']]
    df = df.iloc[1:,:]
    return df
import regex as re

feet_pattern = '^\d'
inegi = 'INEGI'


def din_pob_adapt2(mun : str):
    df2 = base_format(mun, 4)
    df2.columns = df2.iloc[1,:]
    df2 = df2.iloc[2:, :]
    index = list(df2.index)
    extra_poblation = []  # Información Adicional
    for i in index:
        if re.findall(feet_pattern, i):
            extra_poblation.append(i)
            df2 = df2.drop(index = i)
        if re.findall(inegi, i):
            df2 = df2.drop(index = i)
    extra_poblation[0] += 'y la otra mitad por encima.'
    extra_poblation[1] += '(15 a 64 años)'
    pob2 = df2.iloc[:-2,:2]
    return pob2






Dinámica poblacional (2020),Absoluto,Porcentaje
Unnamed: 1,,
Población masculina,1336,52.1
Población femenina,1227,47.9
Población de 0 a 14 años,589,23
Población de 15 a 64 años,1624,63.4
Población de 65 años y más,350,13.7
Población en localidades de 1 a 2 499 habitantes,2563,100
Población en localidades de 2 500 y más habitantes,0,0
Edad mediana 1,34,-
Razón de dependencia 2,57.820197,-


In [ ]:
c0 = ['Población Total','Tasa de Crecimiento Poblacional', 'Indice', 'Migración']
c1_pob = ['Año', 'Sexo', 'Edad', 'Localidades']
c1_values = {
    'Año' : ['1990','2000','2010','2020'],
    'Sexo': ['Masculino', 'Fememino'],
    'Edad': ['0-14', '15-64','65+'],
    'Localidades': ['1-2499','+2500']
}
clast = ['Absoluto', 'Porcentaje']

pob_two = [c for cat in c1_pob for c in c1_values[cat] for j in range(len(clast))]
pob_one = [c1 for c1 in c1_pob for j in range(len(c1_values[c1]) * len(clast))]
pob_three = [c for j in range(len(pob_two) // 2) for c in clast ]
pob_zero = [c0[0] for j in range(len(pob_two))]

mi_pob = pd.MultiIndex.from_arrays([pob_zero, pob_one, pob_two, pob_three])

df_pob = pd.DataFrame(index = Municipios, columns=mi_pob)
df_pob.head()